# Document Transformers

In [1]:
from langchain_text_splitters import CharacterTextSplitter

text = """The world of street food is a vibrant tapestry of flavors and cultures. From sizzling skewers to spicy tacos, every corner of the globe has its own unique offerings. Vendors often  up shop in bustling markets or on busy street corners, attracting hungry passersby with mouthwatering aromas.

One of the best things about street food is its accessibility. It’s quick, affordable, and often made fresh right in front of you. Whether it’s a steaming bowl of pho in Vietnam or a crispy samosa in India, there’s something for everyone to enjoy.

Street food also brings people together. Friends and families gather around food stalls, sharing dishes and stories. It’s a social experience that transcends language and culture, creating connections over a shared love of good eats.

Finally, street food is constantly evolving. Chefs are experimenting with traditional recipes, adding modern twists and fusion flavors. This creativity keeps the scene exciting and ensures there’s always something new to try."""



## Length Based Splitter

- Token-based: Splits text based on the number of tokens, which is useful when working with language models.
- Character-based: Splits text based on the number of characters, which can be more consistent across different types of text.




In [7]:

# text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=300, chunk_overlap=10)
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base", chunk_size=100, chunk_overlap=20
)


docs = text_splitter.create_documents([text])

print(type(docs))

print(len(docs))

print(docs[0])


<class 'list'>
3
page_content='The world of street food is a vibrant tapestry of flavors and cultures. From sizzling skewers to spicy tacos, every corner of the globe has its own unique offerings. Vendors often  up shop in bustling markets or on busy street corners, attracting hungry passersby with mouthwatering aromas.'


## Text-strucutre based splitter

- The RecursiveCharacterTextSplitter attempts to keep larger units (e.g., paragraphs) intact.
- If a unit exceeds the chunk size, it moves to the next level (e.g., sentences).
- This process continues down to the word level if necessary.

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=10)

docs = text_splitter.create_documents([text])

print(type(docs))

print(len(docs))

print(docs[0])

<class 'list'>
8
page_content='The world of street food is a vibrant tapestry of flavors and cultures. From sizzling skewers to spicy tacos, every corner of the globe has its own unique offerings. Vendors often  up shop in bustling'


## Document-structured based
- Preserves the logical organization of the document
- Maintains context within each chunk
- Can be more effective for downstream tasks like retrieval or summarization

**Examples of structure-based splitting:**
- Markdown: Split based on headers (e.g., #, ##, ###)
- HTML: Split using tags
- JSON: Split by object or array elements
- Code: Split by functions, classes, or logical blocks



## Semantic Meaning Based

- Start with the first few sentences and generate an embedding.
- Move to the next group of sentences and generate another embedding (e.g., using a sliding window approach).
- Compare the embeddings to find significant differences, which indicate potential "break points" between semantic sections.
